In [51]:
import os
import re
import json
import requests
from typing import Any
from collections import defaultdict
from copy import copy

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://fr.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F'
OUTPUT_DIR = 'data/fr'

FRENCH_MONTHS = [
    'Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin',
    'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre'
]
MONTH_NUM_TO_NAME = {i + 1: name for i, name in enumerate(FRENCH_MONTHS)}


def get_year_links_from_archive(main_page_url: str) -> dict[str, dict[str, Any]]:
    """Get the year links from the main archive page."""
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    sibling_div = soup.find('div', class_='mw-heading mw-heading2 ext-discussiontools-init-section')
    if not sibling_div:
        raise ValueError("Could not find the nearest sibling div with class 'mw-heading mw-heading2 ext-discussiontools-init-section'.")
    table = sibling_div.find_next_siblings()[1]
    if not table:
        raise ValueError("Could not find the table.")
    p = table.find('p')
    if not p:
        raise ValueError("Could not find the paragraph.")
    
    archive_links = {}
    for a_tag in p.find_all('a', href=True)[1:]: # skip the modifier link
        year = a_tag.text.strip()
        if re.match(r'^\d{4}$', year):
            href = a_tag['href']
            full_url = urljoin(BASE_URL, href)
            exists = 'new' not in a_tag.get('class', [])
            archive_links[year] = {
                'url': full_url,
                'exists': exists
            }

    return archive_links

In [19]:
archive = get_year_links_from_archive(MAIN_URL)

archive

{'2004': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2004',
  'exists': True},
 '2005': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2005',
  'exists': True},
 '2006': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2006',
  'exists': True},
 '2007': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2007',
  'exists': True},
 '2008': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2008',
  'exists': True},
 '2009': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2009',
  'exists': True},
 '2010': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2010',
  'exists': True},
 '2011': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Le_saviez-vous_%3F/Archives/2011',
  'exists': True},
 '2012': {'url': 'https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:L

In [49]:
def _extract_fact_data(element: Tag, base_url: str) -> dict[str, Any]:
    """Extracts text, links, and relevant links from a BeautifulSoup Tag."""
    # Create a copy of the element to safely remove unwanted tags for text extraction.
    text_element = copy(element)
    for tag_to_remove in text_element.find_all(['figure', 'dl']):
        tag_to_remove.decompose()
        
    fact_text = text_element.get_text(" ", strip=True).replace('\xa0', ' ')

    links = []
    relevant_links = []
    for a in element.find_all("a", href=True):
        if a.find_parent(['figure', 'dl']):
            continue

        href = a["href"]
        if not href.startswith("/wiki/"):
            continue

        full_url = unquote(urljoin(base_url, href))

        if a.find_parent('b'):
            relevant_links.append(full_url)

        links.append(full_url)

    dl_tag = element.find('dl')
    section_from_dl = None
    if dl_tag:
        dl_text = dl_tag.get_text()
        
        # Format 1: YYYY-MM-DD
        match_ymd = re.search(r'(\d{4})-(\d{2})-(\d{2})', dl_text)
        if match_ymd:
            y, m, d = match_ymd.groups()
            month_name = MONTH_NUM_TO_NAME.get(int(m))
            if month_name:
                section_from_dl = f"{int(d):02d} {month_name} {y}"
        
        # Format 2: DD month_name YYYY
        else:
            month_pattern = '|'.join(FRENCH_MONTHS)
            match_dmy = re.search(r'(\d{1,2})\s+(' + month_pattern + r')\s+(\d{4})', dl_text, re.IGNORECASE)
            if match_dmy:
                d, month_name, y = match_dmy.groups()
                month_name_capitalized = next((m for m in FRENCH_MONTHS if m.lower() == month_name.lower()), month_name)
                section_from_dl = f"{int(d):02d} {month_name_capitalized} {y}"

    return {
        "text": fact_text,
        "links": links,
        "relevant_links": relevant_links,
        "section_from_dl": section_from_dl,
    }


def parse_year_facts(year_url: str, year: str) -> list[dict]:
    """Parse the facts from the year page."""
    resp = requests.get(year_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    # Parse facts from all uls in section div
    div = soup.find("div", class_="mw-content-ltr mw-parser-output")
    uls = div.find_all("ul")

    for ul in uls:
        for li in ul.find_all("li"):
            fact_data = _extract_fact_data(li, year_url)
            section = fact_data.pop('section_from_dl', None) or year
            if fact_data['text']:
                results.append({
                    "section": section,
                    **fact_data
                })

    return results

In [59]:
year = '2025'
year_url = archive[year]['url']

facts = parse_year_facts(year_url, year)
facts

[{'section': '01 Janvier 2025',
  'text': 'Oublié par le roi Arthur , Ségurant (illustration) a également été oublié ou négligé durant de nombreux siècles.',
  'links': ['https://fr.wikipedia.org/wiki/Roi_Arthur',
   'https://fr.wikipedia.org/wiki/Ségurant,_le_chevalier_au_dragon'],
  'relevant_links': ['https://fr.wikipedia.org/wiki/Ségurant,_le_chevalier_au_dragon']},
 {'section': '02 Janvier 2025',
  'text': 'Michael Caine était sur le point de se rendre au Brésil pour rencontrer une fille « belle » et « extraordinaire » découverte dans une publicité locale lorsqu’il a appris qu’elle habitait le quartier londonien de Fulham à 2 km de chez lui.',
  'links': ['https://fr.wikipedia.org/wiki/Michael_Caine',
   'https://fr.wikipedia.org/wiki/Brésil',
   'https://fr.wikipedia.org/wiki/Shakira_Caine',
   'https://fr.wikipedia.org/wiki/Fulham'],
  'relevant_links': ['https://fr.wikipedia.org/wiki/Shakira_Caine']},
 {'section': '03 Janvier 2025',
  'text': 'Ayant fait partie de la première p

In [66]:
def _extract_year_and_month_from_section(section: str) -> tuple[str, str]:
    """Extracts year and month from the section string."""
    month_pattern = '|'.join(FRENCH_MONTHS)
    # Match "DD Month YYYY"
    match_dmy = re.match(r'\d{1,2}\s+(' + month_pattern + r')\s+(\d{4})', section, re.IGNORECASE)
    if match_dmy:
        month, year = match_dmy.groups()
        month_capitalized = next((m for m in FRENCH_MONTHS if m.lower() == month.lower()), month)

        return year, month_capitalized

    # Match "YYYY"
    year = section.strip()
    if year.isdigit():
        return year, "Janvier"

    raise ValueError(f"Could not extract year and month from section: {section}")

_extract_year_and_month_from_section(' 2025')

('2025', 'Janvier')